## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-11-17-51-00 +0000,48hills,"Under the Stars: Superlative locals Umbrellas,...",https://48hills.org/2025/09/under-the-stars-su...
1,2025-09-11-17-47-51 +0000,nyt,Klarna’s Market Debut Shows Wall Street’s Rene...,https://www.nytimes.com/2025/09/11/business/de...
2,2025-09-11-17-47-00 +0000,wsj,"Canada to Champion Mining, Energy and Port Pro...",https://www.wsj.com/world/americas/canada-to-c...
3,2025-09-11-17-42-44 +0000,nyt,Mamdani Gets Backing From National Democrats a...,https://www.nytimes.com/2025/09/11/us/politics...
4,2025-09-11-17-42-34 +0000,nyt,FBI Releases Photos of Person of Interest in C...,https://www.nytimes.com/2025/09/11/us/fbi-phot...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2304/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
39,kirk,77
38,charlie,70
30,trump,35
40,shooting,25
98,new,21


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
255,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...,241
196,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...,230
317,2025-09-10-20-36-13 +0000,nyt,Trump and Leaders Across Political Spectrum Re...,https://www.nytimes.com/2025/09/10/us/charlie-...,229
358,2025-09-10-19-03-28 +0000,latimes,Shooting of Charlie Kirk was 'political assass...,https://www.latimes.com/california/story/2025-...,218
146,2025-09-11-12-11-58 +0000,latimes,Charlie Kirk shooting reignites campus free sp...,https://www.latimes.com/california/story/2025-...,217


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
255,241,2025-09-11-00-08-00 +0000,wsj,Charlie Kirk was a provocative conservative in...,https://www.wsj.com/politics/charlie-kirk-turn...
264,76,2025-09-10-23-40-09 +0000,nypost,UK ambassador Peter Mandelson told Jeffrey Eps...,https://nypost.com/2025/09/10/us-news/uk-ambas...
121,69,2025-09-11-13-50-47 +0000,latimes,FBI releases photos of 'person of interest' in...,https://www.latimes.com/world-nation/story/202...
316,67,2025-09-10-20-37-00 +0000,wsj,Nearly half a day after NATO ordered jet fight...,https://www.wsj.com/politics/national-security...
196,59,2025-09-11-09-00-00 +0000,wsj,The killing of the conservative activist Charl...,https://www.wsj.com/politics/political-violenc...
110,46,2025-09-11-14-17-00 +0000,bbc,Suspect accused of killing elderly couple in N...,https://www.bbc.com/news/articles/c4gkxddzzeyo...
275,44,2025-09-10-23-09-00 +0000,wsj,A U.S. military aircraft struck a boat suspect...,https://www.wsj.com/politics/national-security...
183,43,2025-09-11-10-00-00 +0000,latimes,Beloved SoCal octopus will spend her final day...,https://www.latimes.com/california/story/2025-...
318,42,2025-09-10-20-33-00 +0000,wsj,Inside NATO’s Scramble to Shoot Down Russia’s ...,https://www.wsj.com/world/europe/inside-natos-...
285,37,2025-09-10-22-31-00 +0000,wsj,Mexico plans to raise tariffs on imports of ke...,https://www.wsj.com/economy/trade/mexico-to-im...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
